In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [2]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Chuẩn hoá Max Min
scaler = MinMaxScaler()
scaler.fit_transform(train)
train_scaled = scaler.transform(train)
train_scaled_df = pd.DataFrame(train_scaled, columns=data.columns, index=train.index)
test_scaled = scaler.transform(test)
test_scaled_df = pd.DataFrame(test_scaled, columns=data.columns, index=test.index)

NameError: name 'data' is not defined

In [ ]:
# Kiểm tra tính dừng của các chuỗi (ADF test)
from statsmodels.tsa.stattools import adfuller

def adf_test(series, title=''):

    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(out.to_string())
    print('')

# Áp dụng ADF test cho từng đặc trưng
adf_test(train['SoLuong'], title= 'SoLuong')

Augmented Dickey-Fuller Test: FDIGiaiNgan
ADF Test Statistic      -2.941180
p-value                  0.040765
# Lags Used              9.000000
# Observations Used     18.000000
Critical Value (1%)     -3.859073
Critical Value (5%)     -3.042046
Critical Value (10%)    -2.660906



Dùng tách mùa

In [9]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose

# Hàm tính toán các độ đo
def calculate_metrics(true_values, forecast_values):
    mse = np.mean((true_values - forecast_values) ** 2)  # Mean Squared Error
    mae = np.mean(np.abs(true_values - forecast_values))  # Mean Absolute Error
    rmse = np.sqrt(mse) 
    mape = np.mean(np.abs((true_values - forecast_values) / true_values)) * 100  # Mean Absolute Percentage Error
    return mse, mae, rmse, mape

def print_metrics(true_values, forecast_values):
    mse, mae, rmse, mape = calculate_metrics(true_values, forecast_values)
    print(f'MSE: {np.round(mse, 5)}, MAE: {np.round(mae, 5)}, RMSE: {np.round(rmse, 5)}, MAPE: {np.round(mape, 5)}')

def forecast_var(data, forecast_steps ,max_lag ,Period):
    # Tách mùa cho từng chuỗi dữ liệu
    seasonal_decomposed = {}
    for column in data.columns:
        decomposition = seasonal_decompose(data[column], model='additive', period=Period)
        seasonal_decomposed[column] = decomposition.seasonal

    # Trừ phần mùa khỏi dữ liệu gốc để chỉ còn lại phần xu hướng và ngẫu nhiên
    deseasonalized_data = data - pd.DataFrame(seasonal_decomposed)

    # Chia tập train và tập test
    train_size = int(len(deseasonalized_data) * 0.8)  # Sử dụng 80% dữ liệu làm tập train
    train, test = deseasonalized_data.iloc[:train_size], deseasonalized_data.iloc[train_size:]

    # Chuẩn hoá dữ liệu bằng Min-Max scaler chỉ trên tập huấn luyện
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train)
    train_scaled_df = pd.DataFrame(train_scaled, columns=data.columns, index=train.index)
    test_scaled = scaler.transform(test)
    test_scaled_df = pd.DataFrame(test_scaled, columns=data.columns, index=test.index)

    # Áp dụng sai phân
    train_diff_df = train_scaled_df.diff().dropna()
    test_diff_df = test_scaled_df.diff().dropna()

    # Xây dựng mô hình VAR trên tập train
    model = VAR(train_diff_df)

    # Tìm ra số lượng lag tốt nhất
    # best_lag = model.select_order()
    # best_lag = best_lag.selected_orders['aic']

    # Fit mô hình với số lượng lag tốt nhất trên tập train
    result = model.fit(max_lag)

    # Dự báo trên tập test
    forecast = result.forecast(train_diff_df.values[-max_lag:], steps=len(test_diff_df))

    # Chuyển kết quả dự báo về dạng DataFrame
    forecast_df = pd.DataFrame(forecast, columns=data.columns, index=test_diff_df.index)

    # Đảo ngược quá trình sai phân để có kết quả dự báo thực tế
    forecast_df = forecast_df.cumsum()
    forecast_df = pd.DataFrame(scaler.inverse_transform(forecast_df), columns=data.columns, index=test_diff_df.index)

    # Thêm lại phần mùa vào kết quả dự báo
    seasonal_forecast = pd.DataFrame(seasonal_decomposed).iloc[train_size + 1:train_size + 1 + len(forecast_df)]
    forecast_df += seasonal_forecast.values

    # Cắt bỏ hàng cuối của tập test để phù hợp với độ dài của forecast_df
    test = data[-forecast_steps:]
    test_values = test.iloc[:, 0]
    forecast = forecast_df[-forecast_steps:]
    forecast_values = forecast.iloc[:, 0]
    # print(forecast_values)
    # print(test_values)

    # Tính toán và in ra các độ đo
    print_metrics(test_values, forecast_values.values)


In [15]:
data = pd.read_csv('DL_TongHop.csv')  # Thay 'du_lieu.csv' bằng tên file của bạn
# data = pd.read_excel('DL_Tonghop.xlsx')  # Thay 'du_lieu.csv' bằng tên file của bạn
data.set_index('Thang', inplace=True)

print("Trường hợp Var(1)")
data1 = data[['SoLuong','CPI','XuatKhau','NhapKhau','SxCongNghiep','FDIDangKy','FDIGiaiNgan','BanLe']]
forecast_var(data1, forecast_steps = 6 ,max_lag = 1,Period = 12)

print("Trường hợp Var(2)")
forecast_var(data1, forecast_steps = 6 ,max_lag = 2,Period = 12)

print("Trường hợp Var(3)")
forecast_var(data1, forecast_steps = 6 ,max_lag = 3,Period = 12)

print("Trường hợp Var(4)")
forecast_var(data1, forecast_steps = 6 ,max_lag = 4,Period = 12)

Trường hợp Var(1)
MSE: 39684267834.89459, MAE: 147816.84333, RMSE: 199209.1058, MAPE: 38.66393
Trường hợp Var(2)
MSE: 58586182924.16897, MAE: 180668.17386, RMSE: 242045.82815, MAPE: 59.87996
Trường hợp Var(3)
MSE: 51936889624.3764, MAE: 194783.30903, RMSE: 227896.66436, MAPE: 66.66121
Trường hợp Var(4)
MSE: 35544295607.27241, MAE: 158413.35387, RMSE: 188531.94851, MAPE: 52.30694


c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [16]:
data = pd.read_csv('DL_TongHop.csv')  # Thay 'du_lieu.csv' bằng tên file của bạn
# data = pd.read_excel('DL_Tonghop.xlsx')  # Thay 'du_lieu.csv' bằng tên file của bạn
data.set_index('Thang', inplace=True)

print("Trường hợp Var(1)")
data1 = data[['SoLuong','CPI','XuatKhau','NhapKhau','SxCongNghiep','FDIDangKy','FDIGiaiNgan','BanLe']]
forecast_var(data1, forecast_steps = 3 ,max_lag = 1,Period = 12)

print("Trường hợp Var(2)")
forecast_var(data1, forecast_steps = 3 ,max_lag = 2,Period = 12)

print("Trường hợp Var(3)")
forecast_var(data1, forecast_steps = 3 ,max_lag = 3,Period = 12)

print("Trường hợp Var(4)")
forecast_var(data1, forecast_steps = 3 ,max_lag = 4,Period = 12)

Trường hợp Var(1)
MSE: 19583854052.2233, MAE: 125505.63738, RMSE: 139942.32402, MAPE: 32.58296
Trường hợp Var(2)
MSE: 13897577181.02504, MAE: 97202.47758, RMSE: 117887.98574, MAPE: 29.54838
Trường hợp Var(3)
MSE: 23309585576.47671, MAE: 138716.8125, RMSE: 152674.7706, MAPE: 38.93445
Trường hợp Var(4)
MSE: 14251882070.29255, MAE: 101764.93001, RMSE: 119381.24673, MAPE: 22.27879


c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\User\anaconda3\envs\tensorFlow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Dùng sai phân

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.ar_model import AutoReg
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose

# Hàm tính toán các độ đo
def calculate_metrics(true_values, forecast_values):
    mse = np.mean((true_values - forecast_values) ** 2)  # Mean Squared Error
    mae = np.mean(np.abs(true_values - forecast_values))  # Mean Absolute Error
    rmse = np.sqrt(mse) 
    mape = np.mean(np.abs((true_values - forecast_values) / true_values)) * 100  # Mean Absolute Percentage Error
    return mse, mae, rmse, mape

def print_metrics(true_values, forecast_values):
    mse, mae, rmse, mape = calculate_metrics(true_values, forecast_values)
    print(f'MSE: {np.round(mse)}, MAE: {np.round(mae)}, RMSE: {np.round(rmse)}, MAPE: {np.round(mape, 5)}%')

def forecast_var(data, forecast_steps ,max_lag ,Period):
    # Tách mùa cho từng chuỗi dữ liệu
    seasonal_decomposed = {}
    for column in data.columns:
        decomposition = seasonal_decompose(data[column], model='additive', period=Period)
        seasonal_decomposed[column] = decomposition.seasonal

    # Trừ phần mùa khỏi dữ liệu gốc để chỉ còn lại phần xu hướng và ngẫu nhiên
    deseasonalized_data = data - pd.DataFrame(seasonal_decomposed)

    # Chia tập train và tập test
    train_size = int(len(deseasonalized_data) * 0.8)  # Sử dụng 80% dữ liệu làm tập train
    
    train, test = deseasonalized_data.iloc[:train_size], deseasonalized_data.iloc[train_size:]
    # print(test)

    # Chuẩn hoá dữ liệu bằng Min-Max scaler chỉ trên tập huấn luyện
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train)
    train_scaled_df = pd.DataFrame(train_scaled, columns=data.columns, index=train.index)
    test_scaled = scaler.transform(test)
    test_scaled_df = pd.DataFrame(test_scaled, columns=data.columns, index=test.index)

    # # Áp dụng sai phân
    # train_diff_df = train_scaled_df.diff().dropna()
    # test_diff_df = test_scaled_df.diff().dropna()

    # Xây dựng mô hình VAR trên tập train
    model = VAR(train_scaled_df)

    # Tìm ra số lượng lag tốt nhất
    # best_lag = model.select_order()
    # best_lag = best_lag.selected_orders['aic']

    # Fit mô hình với số lượng lag tốt nhất trên tập train
    result = model.fit(max_lag)

    # Dự báo trên tập test
    forecast = result.forecast(train_scaled_df.values[-max_lag:], steps=len(test_scaled_df))

    # Chuyển kết quả dự báo về dạng DataFrame
    forecast_df = pd.DataFrame(forecast, columns=data.columns, index=test_scaled_df.index)

    # Đảo ngược quá trình sai phân để có kết quả dự báo thực tế
    
    forecast_df = pd.DataFrame(scaler.inverse_transform(forecast_df), columns=data.columns, index=test_scaled_df.index)

    # Thêm lại phần mùa vào kết quả dự báo
    seasonal_forecast = pd.DataFrame(seasonal_decomposed).iloc[train_size:]
    # print(seasonal_forecast.values)
    # print(forecast_df)
    forecast_df += seasonal_forecast.values

    # Cắt bỏ hàng cuối của tập test để phù hợp với độ dài của forecast_df
    test = data[-forecast_steps:]
    test_values = test.iloc[:, 0]
    forecast = forecast_df[-forecast_steps:]
    forecast_values = forecast.iloc[:, 0]
    # print(forecast_values)
    # print(test_values)

    # Tính toán và in ra các độ đo
    print_metrics(test_values, forecast_values.values)

In [32]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from sklearn.preprocessing import StandardScaler

def calculate_metrics(true_values, forecast_values):
    mse = np.mean((true_values - forecast_values) ** 2)  # Mean Squared Error
    mae = np.mean(np.abs(true_values - forecast_values))  # Mean Absolute Error
    rmse = np.sqrt(mse) 
    mape = np.mean(np.abs((true_values - forecast_values) / true_values)) * 100  # Mean Absolute Percentage Error
    return mse, mae, rmse, mape

def print_metrics(true_values, forecast_values):
    mse, mae, rmse, mape = calculate_metrics(true_values, forecast_values)
    print(f'MSE: {np.round(mse)}, MAE: {np.round(mae)}, RMSE: {np.round(rmse)}, MAPE: {np.round(mape, 5)}%')

def forecast_ar(ar_series, forecast_steps, lag, period):
    res_ar = sm.tsa.seasonal_decompose(ar_series, period=period, model='additive')
    
    # Deseasonalized data
    deseasonalized_ar = ar_series - res_ar.seasonal
    train_size = int(len(deseasonalized_ar) * 0.8)  # Sử dụng 80% dữ liệu làm tập train
    
    train, test = deseasonalized_ar.iloc[:train_size], deseasonalized_ar.iloc[train_size:]

    # Chuẩn hoá dữ liệu bằng Min-Max scaler chỉ trên tập huấn luyện
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    train_scaled_df = pd.DataFrame(train_scaled, index=train.index)
    test_scaled = scaler.transform(test.values.reshape(-1, 1))
    test_scaled_df = pd.DataFrame(test_scaled, index=test.index)

    # Step 2: Apply AR to the deseasonalized data
    model = AutoReg(train_scaled_df.values.flatten(), lags=lag)
    results = model.fit()

    # Step 3: Forecast using AR model
    forecast = results.predict(start=len(train_scaled_df), end=len(train_scaled_df) + len(test_scaled_df) - 1, dynamic=False)

    # Chuyển kết quả dự báo về dạng DataFrame
    forecast_df = pd.DataFrame(forecast, index=test_scaled_df.index)

    # Đảo ngược quá trình chuẩn hóa để có kết quả dự báo thực tế
    forecast_df = pd.DataFrame(scaler.inverse_transform(forecast_df), index=test_scaled_df.index)

    # Thêm lại phần mùa vào kết quả dự báo
    forecast_df = forecast_df.squeeze() + res_ar.seasonal[train_size:]

    # Cắt bỏ hàng cuối của tập test để phù hợp với độ dài của forecast_df
    test = ar_series.iloc[train_size:]
    test_values = test[-forecast_steps:]
    forecast_values = forecast_df[-forecast_steps:]

    # Tính toán và in ra các độ đo
    print_metrics(test_values.values, forecast_values.values)

# Example usage:
# ar_series should be a pandas Series containing your time series data
# forecast_steps is the number of steps you want to forecast
# lag is the number of lag observations to use in the AR model
# period is the period of seasonality

# Assuming 'data' is your DataFrame and 'SoLuong' is the column with the time series data
# forecast_ar(data['SoLuong'], forecast_steps=10, lag=5, period=12)

    
    
    



In [34]:
data = pd.read_csv('DL_TongHop.csv')  # Thay 'du_lieu.csv' bằng tên file của bạn
# data = pd.read_excel('DL_Tonghop.xlsx')  # Thay 'du_lieu.csv' bằng tên file của bạn
data.set_index('Thang', inplace=True)


data1 = data['SoLuong']

forecast_ar(data1, forecast_steps = 3 ,lag = 1,period = 12)

MSE: 4886201316.0, MAE: 61817.0, RMSE: 69901.0, MAPE: 13.1599%
